In [47]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



Data Collection

In [48]:
kaggle_dictionary = json.load(open("kaggle.json"))
#setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]




In [49]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [50]:
data = pd.read_csv("/content/IMDB Dataset.csv")
data.shape
data.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [51]:
data["sentiment"].value_counts()
data.replace({
    "sentiment" : {"positive":1, "negative": 0}
},inplace = True)
data.head()

<ipython-input-51-f56095c656eb>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [52]:
#split data into training data and test data
train_data,test_data = train_test_split(data,test_size= 0.2, random_state=42)
print(train_data.shape)
print(test_data.shape)


(40000, 2)
(10000, 2)


data preprocessing

In [53]:
#tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen=200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen=200)




In [54]:
print(x_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [55]:
print(x_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [56]:
y_train = train_data["sentiment"]
y_test = test_data["sentiment"]

#LSTM Long short term memory
#build model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128,input_length=200))
model.add(LSTM(128,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation="sigmoid"))



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [61]:
#compile the model


model.compile(optimizer="adam" , loss = "binary_crossentropy", metrics = ["accuracy"])
model.fit(x_train,y_train,epochs=5,batch_size=64,validation_split=0.2)



Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 240s 476ms/step - accuracy: 0.7162 - loss: 0.5435 - val_accuracy: 0.7968 - val_loss: 0.4505
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 242s 436ms/step - accuracy: 0.8038 - loss: 0.4406 - val_accuracy: 0.8551 - val_loss: 0.3554
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 265s 442ms/step - accuracy: 0.8247 - loss: 0.3987 - val_accuracy: 0.8482 - val_loss: 0.3606
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 291s 501ms/step - accuracy: 0.8762 - loss: 0.3098 - val_accuracy: 0.8431 - val_loss: 0.3835
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 251s 480ms/step - accuracy: 0.8590 - loss: 0.3333 - val_accuracy: 0.8696 - val_loss: 0.3269


In [64]:
loss,accuracy = model.evaluate(x_test,y_test)
print(f"Test loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 30s 97ms/step - accuracy: 0.8761 - loss: 0.3069
Test loss: 0.30724620819091797
Test Accuracy: 0.8776999711990356


In [68]:
def predict_sentiment(review):
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen = 200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment




In [69]:
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of review is: {sentiment}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
The sentiment of review is: positive


In [70]:
new_review = "This movie was not good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of review is: {sentiment}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
The sentiment of review is: negative


In [71]:
new_review = "This movie was not good and i do not like this movie."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
The sentiment of review is: negative
